# NLP / NLTK - Build a Classification Model on Spam / Ham Text Data
#### Building Machine Learning Classifiers Random Forest Classifier & Gradient Boosting Classifier 

### Import Libraries

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

In [2]:
stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

### Read in & clean text

In [4]:
data = pd.read_csv('SMSSpamCollection.tsv', sep = '\t', header = None)
data.columns = ['label', 'body_text']

data.head()

,label,body_text
0,ham,I've been searching for the right words to tha...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,"Nah I don't think he goes to usf, he lives aro..."
3,ham,Even my brother is not like to speak with me. ...
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!


In [5]:
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text)- text.count(" ")),3) * 100

In [7]:
data['body_len'] = data['body_text'].apply( lambda x : len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

data.head()

,label,body_text,body_len,punct%
0,ham,I've been searching for the right words to tha...,160,2.5
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...,128,4.7
2,ham,"Nah I don't think he goes to usf, he lives aro...",49,4.1
3,ham,Even my brother is not like to speak with me. ...,62,3.2
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!,28,7.1


In [8]:
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

### GenerateTfidf values

In [9]:
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])

In [10]:
X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis =1)

X_features.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8097,8098,8099,8100,8101,8102,8103,8104,8105,8106
0,160,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Explore RandomForestClassifier Attributes & Hyperparameters

In [11]:
from sklearn.ensemble import RandomForestClassifier

In [12]:
print(dir(RandomForestClassifier))

print(RandomForestClassifier())

['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_cache', '_abc_negative_cache', '_abc_negative_cache_version', '_abc_registry', '_estimator_type', '_get_param_names', '_make_estimator', '_set_oob_score', '_validate_X_predict', '_validate_estimator', '_validate_y_class_weight', 'apply', 'decision_path', 'feature_importances_', 'fit', 'get_params', 'predict', 'predict_log_proba', 'predict_proba', 'score', 'set_params']
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0,

### Explore RandomForestClassifier through Cross-Validation

In [13]:
from sklearn.model_selection import KFold, cross_val_score

In [14]:
rf = RandomForestClassifier(n_jobs=-1)

k_fold = KFold(n_splits = 5)

cross_val_score(rf, X_features, data['label'], cv = k_fold, scoring='accuracy', n_jobs=-1)

array([0.97037702, 0.97217235, 0.97127469, 0.96316262, 0.97214735])

### Explore RandomForestClassifier through Holdout Set

In [15]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size = 0.2)

In [18]:
from sklearn.ensemble import RandomForestClassifier

In [21]:
rf = RandomForestClassifier(n_estimators=50, max_depth=20, n_jobs=-1)
rf_model = rf.fit(X_train, y_train)

In [22]:
# Top 10 important features

sorted(zip(rf_model.feature_importances_, X_train.columns), reverse=True)[0:10] 

# zip will combine 2 arrays and sorted will dort that combined arrray


[(0.05447616276421086, 'body_len'),
 (0.05112391472606328, 1804),
 (0.025840624542984716, 7353),
 (0.023746741582591987, 3135),
 (0.021812538331816955, 4799),
 (0.021416115934430064, 5727),
 (0.019967643227692466, 7030),
 (0.019338624451306245, 5991),
 (0.016003494519163428, 7221),
 (0.014722596445981753, 1882)]

In [25]:
y_pred = rf_model.predict(X_test)

precision, recall, fscore, support = score(y_test, y_pred, pos_label ='spam', average='binary')

In [26]:
print('Precision: {} / Recall: {} / Accuracy: {}'.format(round(precision, 3),
                                                        round(recall, 3),
                                                        round((y_pred==y_test).sum() / len(y_pred),3)))

Precision: 1.0 / Recall: 0.587 / Accuracy: 0.943


# Building Machine Learning Classifiers: Explore Random Forest model with grid-search

### Build our own Grid-search

In [31]:
def train_RF(n_est, depth):
    rf = RandomForestClassifier(n_estimators=n_est, max_depth=depth, n_jobs=-1)
    rf_model = rf.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)
    precision, recall, fscore, support = score(y_test, y_pred, pos_label ='spam', average='binary')
    print('Est: {} / Depth: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
        n_est, depth, round(precision, 3), round(recall, 3),
        round((y_pred==y_test).sum() / len(y_pred), 3)))

In [32]:
for n_est in [10, 50, 100]:
    for depth in[10, 20, 30, None]:
        train_RF(n_est, depth)

Est: 10 / Depth: 10 ---- Precision: 1.0 / Recall: 0.258 / Accuracy: 0.897
Est: 10 / Depth: 20 ---- Precision: 1.0 / Recall: 0.587 / Accuracy: 0.943
Est: 10 / Depth: 30 ---- Precision: 1.0 / Recall: 0.639 / Accuracy: 0.95
Est: 10 / Depth: None ---- Precision: 0.984 / Recall: 0.806 / Accuracy: 0.971
Est: 50 / Depth: 10 ---- Precision: 1.0 / Recall: 0.258 / Accuracy: 0.897
Est: 50 / Depth: 20 ---- Precision: 1.0 / Recall: 0.632 / Accuracy: 0.949
Est: 50 / Depth: 30 ---- Precision: 1.0 / Recall: 0.723 / Accuracy: 0.961
Est: 50 / Depth: None ---- Precision: 1.0 / Recall: 0.826 / Accuracy: 0.976
Est: 100 / Depth: 10 ---- Precision: 1.0 / Recall: 0.213 / Accuracy: 0.89
Est: 100 / Depth: 20 ---- Precision: 1.0 / Recall: 0.6 / Accuracy: 0.944
Est: 100 / Depth: 30 ---- Precision: 1.0 / Recall: 0.71 / Accuracy: 0.96
Est: 100 / Depth: None ---- Precision: 0.985 / Recall: 0.839 / Accuracy: 0.976


### Exploring parameter settings using GridSearchCV

In [36]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

import datetime

In [37]:
start = datetime.datetime.now()

rf = RandomForestClassifier()
param ={'n_estimators': [10,50,150,300],
        'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf, param, cv =5, n_jobs =-1)

gs_fit = gs.fit(X_features, data['label'])

pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending =False)[0:5]
            
end = datetime.datetime.now()

elapsed = end - start

print('Grid Search Time: ', elapsed )

/anaconda/lib/python3.6/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Grid Search Time:  0:04:11.697680


/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125

In [38]:
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending =False)[0:5]

/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
13,6.639183,0.525301,0.131882,0.039224,None,50,"{'max_depth': None, 'n_estimators': 50}",0.977578,0.977558,0.973944,...,0.975216,0.002771,1,1.000000,1.000000,0.999776,0.999776,0.999327,0.999776,0.000246
10,13.860476,0.161678,0.179504,0.044912,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.978475,0.979354,0.973944,...,0.974856,0.003866,2,0.999326,0.999551,0.999102,0.999551,0.998878,0.999282,0.000262
8,1.712217,0.025448,0.048997,0.001511,90,10,"{'max_depth': 90, 'n_estimators': 10}",0.974888,0.967684,0.976640,...,0.973420,0.005464,3,0.997979,0.998428,0.998429,0.998878,0.997980,0.998339,0.000336
5,4.970313,0.371540,0.073239,0.014312,60,50,"{'max_depth': 60, 'n_estimators': 50}",0.977578,0.975763,0.973944,...,0.973240,0.003298,4,0.994161,0.991693,0.994388,0.992817,0.993715,0.993355,0.000990
15,31.302471,5.575668,0.227706,0.080773,None,300,"{'max_depth': None, 'n_estimators': 300}",0.975785,0.977558,0.973944,...,0.973240,0.003345,4,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000


# Building Machine Learning Classifiers: Explore Gradient Boosting model with grid-search

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

### Explore GradientBoostingClassifier Attributes & Hyperparameters

In [40]:
from sklearn.ensemble import GradientBoostingClassifier

In [41]:
print(dir(GradientBoostingClassifier))

print(GradientBoostingClassifier)

['_SUPPORTED_LOSS', '__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_cache', '_abc_negative_cache', '_abc_negative_cache_version', '_abc_registry', '_check_initialized', '_check_params', '_clear_state', '_decision_function', '_estimator_type', '_fit_stage', '_fit_stages', '_get_param_names', '_init_decision_function', '_init_state', '_is_initialized', '_make_estimator', '_resize_state', '_staged_decision_function', '_validate_estimator', '_validate_y', 'apply', 'decision_function', 'feature_importances_', 'fit', 'get_params', 'n_features', 'predict', 'predict_log_proba', 'predict_proba', 'score', 's

### Build our own Grid-search

In [42]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)

In [44]:
def train_GB(est, max_depth, lr):
    gb = GradientBoostingClassifier(n_estimators=est, max_depth=max_depth, learning_rate=lr)
    gb_model = gb.fit(X_train, y_train)
    y_pred = gb_model.predict(X_test)
    precision, recall, fscore, train_support = score(y_test, y_pred, pos_label='spam', average='binary')
    print('Est: {} / Depth: {} / LR: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
        est, max_depth, lr, round(precision, 3), round(recall, 3), 
        round((y_pred==y_test).sum()/len(y_pred), 3)))

In [45]:
for n_est in [50, 100, 150]:
    for max_depth in [3, 7, 11, 15]:
        for lr in [0.01, 0.1, 1]:
            train_GB(n_est, max_depth, lr)

/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Est: 50 / Depth: 3 / LR: 0.01 ---- Precision: 0.0 / Recall: 0.0 / Accuracy: 0.864
Est: 50 / Depth: 3 / LR: 0.1 ---- Precision: 0.943 / Recall: 0.757 / Accuracy: 0.961
Est: 50 / Depth: 3 / LR: 1 ---- Precision: 0.886 / Recall: 0.77 / Accuracy: 0.955


/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Est: 50 / Depth: 7 / LR: 0.01 ---- Precision: 0.0 / Recall: 0.0 / Accuracy: 0.864
Est: 50 / Depth: 7 / LR: 0.1 ---- Precision: 0.906 / Recall: 0.822 / Accuracy: 0.964
Est: 50 / Depth: 7 / LR: 1 ---- Precision: 0.872 / Recall: 0.809 / Accuracy: 0.958
Est: 50 / Depth: 11 / LR: 0.01 ---- Precision: 1.0 / Recall: 0.007 / Accuracy: 0.864
Est: 50 / Depth: 11 / LR: 0.1 ---- Precision: 0.914 / Recall: 0.836 / Accuracy: 0.967
Est: 50 / Depth: 11 / LR: 1 ---- Precision: 0.846 / Recall: 0.829 / Accuracy: 0.956


/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Est: 50 / Depth: 15 / LR: 0.01 ---- Precision: 0.0 / Recall: 0.0 / Accuracy: 0.864
Est: 50 / Depth: 15 / LR: 0.1 ---- Precision: 0.888 / Recall: 0.836 / Accuracy: 0.963
Est: 50 / Depth: 15 / LR: 1 ---- Precision: 0.867 / Recall: 0.855 / Accuracy: 0.962
Est: 100 / Depth: 3 / LR: 0.01 ---- Precision: 0.966 / Recall: 0.566 / Accuracy: 0.938
Est: 100 / Depth: 3 / LR: 0.1 ---- Precision: 0.938 / Recall: 0.796 / Accuracy: 0.965
Est: 100 / Depth: 3 / LR: 1 ---- Precision: 0.894 / Recall: 0.776 / Accuracy: 0.957
Est: 100 / Depth: 7 / LR: 0.01 ---- Precision: 0.917 / Recall: 0.658 / Accuracy: 0.945
Est: 100 / Depth: 7 / LR: 0.1 ---- Precision: 0.913 / Recall: 0.829 / Accuracy: 0.966
Est: 100 / Depth: 7 / LR: 1 ---- Precision: 0.882 / Recall: 0.789 / Accuracy: 0.957
Est: 100 / Depth: 11 / LR: 0.01 ---- Precision: 0.885 / Recall: 0.757 / Accuracy: 0.953
Est: 100 / Depth: 11 / LR: 0.1 ---- Precision: 0.927 / Recall: 0.836 / Accuracy: 0.969
Est: 100 / Depth: 11 / LR: 1 ---- Precision: 0.854 / Recal

### Exploring parameter settings using GridSearchCV

In [48]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [50]:
start = datetime.datetime.now()

gb = GradientBoostingClassifier()
param = {
    'n_estimators': [100, 150], 
    'max_depth': [7, 11, 15],
    'learning_rate': [0.1]
}

clf = GridSearchCV(gb, param, cv=5, n_jobs=-1)
cv_fit = clf.fit(X_train, y_train)

end = datetime.datetime.now()

elapsed = end - start
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

/anaconda/lib/python3.6/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: 

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
1,141.191128,2.987139,0.120720,0.015863,0.1,7,150,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.969697,0.971942,...,0.969690,0.002924,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0
5,257.799753,38.779985,0.104996,0.014928,0.1,15,150,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.969697,0.966330,...,0.968792,0.003709,2,1.0,1.0,1.0,1.0,1.0,1.0,0.0
3,208.677667,4.630305,0.111481,0.009201,0.1,11,150,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.969697,0.965208,...,0.968343,0.003119,3,1.0,1.0,1.0,1.0,1.0,1.0,0.0
0,98.617326,1.281452,0.163775,0.031161,0.1,7,100,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.966330,0.967452,...,0.966996,0.002969,4,1.0,1.0,1.0,1.0,1.0,1.0,0.0
2,157.033042,1.997759,0.113320,0.014398,0.1,11,100,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.967452,0.964085,...,0.966996,0.003852,4,1.0,1.0,1.0,1.0,1.0,1.0,0.0


# Building Machine Learning Classifiers: Model selection

### Read in & clean text

In [51]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

### Split into train/test

In [52]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data[['body_text', 'body_len', 'punct%']], data['label'], test_size=0.2)

### Vectorize text

In [53]:
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
tfidf_vect_fit = tfidf_vect.fit(X_train['body_text']) # IMP fit will only store the vectorized object words

tfidf_train = tfidf_vect_fit.transform(X_train['body_text']) # IMP fit.transform will change the train
tfidf_test = tfidf_vect_fit.transform(X_test['body_text']) 
# IMP fit.transform will change the test using only those vectorized columns form train, tfidf_train & tfidf_test will 
# have same no of columns.

X_train_vect = pd.concat([X_train[['body_len', 'punct%']].reset_index(drop=True), 
           pd.DataFrame(tfidf_train.toarray())], axis=1)
X_test_vect = pd.concat([X_test[['body_len', 'punct%']].reset_index(drop=True), 
           pd.DataFrame(tfidf_test.toarray())], axis=1)

X_train_vect.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,7205,7206,7207,7208,7209,7210,7211,7212,7213,7214
0,37,21.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,102,3.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,22,27.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,91,7.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,25,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Final evaluation of models

In [54]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import precision_recall_fscore_support as score
import time

In [55]:
rf = RandomForestClassifier(n_estimators=150, max_depth=None, n_jobs=-1)

start = time.time()
rf_model = rf.fit(X_train_vect, y_train)
end = time.time()
fit_time = (end - start)

start = time.time()
y_pred = rf_model.predict(X_test_vect)
end = time.time()
pred_time = (end - start)

precision, recall, fscore, train_support = score(y_test, y_pred, pos_label='spam', average='binary')
print('Fit time: {} / Predict time: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(fit_time, 3), round(pred_time, 3), round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))

Fit time: 4.018 / Predict time: 0.206 ---- Precision: 1.0 / Recall: 0.859 / Accuracy: 0.981


In [56]:
gb = GradientBoostingClassifier(n_estimators=150, max_depth=11)

start = time.time()
gb_model = gb.fit(X_train_vect, y_train)
end = time.time()
fit_time = (end - start)

start = time.time()
y_pred = gb_model.predict(X_test_vect)
end = time.time()
pred_time = (end - start)

precision, recall, fscore, train_support = score(y_test, y_pred, pos_label='spam', average='binary')
print('Fit time: {} / Predict time: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(fit_time, 3), round(pred_time, 3), round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))

Fit time: 173.036 / Predict time: 0.078 ---- Precision: 0.906 / Recall: 0.846 / Accuracy: 0.968
